In [1]:
!pip install -q sentence-transformers pymilvus langchain-community


In [2]:
from pymilvus import MilvusClient, DataType

CLUSTER_ENDPOINT = "https://in03-22706440ab83439.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "fd3df03a845da1e42df2a2975f1d4733f62b044415d021f9f6047ef1f631504509c1a870e1370ebc8587c04883fae0c72c886204"

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

# Define the collection name
collection_name = "document_embeddings"

# Connect to Milvus (ensure you've set up the connection correctly)
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)

In [4]:
# Load your local embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


<ipython-input-4-388e7c865f49>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

In [5]:
# load the documents from Milvus
vector_store = Milvus(
    collection_name=collection_name,
    embedding_function=embeddings,
    connection_args={
        "uri": CLUSTER_ENDPOINT,
        "token": TOKEN,  # omit if unused
    }
)

<ipython-input-5-4e7d03429f39>:2: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vector_store = Milvus(


In [6]:
query = "What are the symptoms of asthma?"
retrieved_docs = vector_store.similarity_search(query, k=3)

print("Top Retrieved Documents:")
for doc in retrieved_docs:
    # print the source
    print(f"Source: {doc.metadata['source']}")
    # print the first 300 characters of the page content
    print(doc.page_content[:300])  # Print a snippet
    print("\n")


Top Retrieved Documents:
Source: C:\Users\weszi\OneDrive\Bureau\pfa\dragin-healthcare\docs\kb_chunks\about_asthma_cdc_chunk1.json
January 22, 2024
EspaÃ±ol
About Asthma
Key points
Asthma is a disease that affects your lungs.
It is one of the most common long-term diseases of children, but adults can have asthma, too.
Asthma causes wheezing, breathlessness, chest tightness, and coughing at night or early in the morning.
Symptom


Source: C:\Users\weszi\OneDrive\Bureau\pfa\dragin-healthcare\docs\kb_chunks\about_asthma_cdc_chunk2.json
Symptoms
Asthma causes wheezing, breathlessness, chest tightness, and coughing at night or early in the morning.  If you have asthma, you have it all the time, but you will have asthma attacks only when something bothers your lungs.


Source: C:\Users\weszi\OneDrive\Bureau\pfa\dragin-healthcare\docs\kb_chunks\about_asthma_cdc_chunk3.json
An asthma attack may include coughing, chest tightness, wheezing, and trouble breathing. The attack happens in your body's

In [7]:
%pip install -q  langchain_openai ctransformers sacremoses


In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# ✅ Install required packages (if not already installed)
!pip install transformers accelerate bitsandbytes

In [10]:
!pip install -U bitsandbytes # Upgrade bitsandbytes to the latest version

In [11]:
# ✅ Imports
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# ✅ Device setup (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Model name
model_name = "microsoft/phi-2"

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Load model in 8-bit precision with automatic device placement
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,       # 8-bit quantization to save RAM
    device_map="auto",       # Automatically map to GPU if available
    torch_dtype="auto"       # Automatically choose best dtype
)

# ✅ Set up the text-generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Using device: cuda


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
# Provide a specific prompt
prompt = "What are the symptoms of asthma? in bullet points"

# Generate text with proper truncation and max length
response = generator(prompt,
                     max_length=200,  # Increase max length
                     num_return_sequences=1,
                     truncation=True,  # Ensure truncation
                     do_sample=True,   # Enable sampling for variety
                     top_p=0.9,        # Control diversity
                     temperature=0.7)  # Control randomness

print("Generated Response:", response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response: What are the symptoms of asthma? in bullet points?
Answer: Symptoms of asthma include:
- Wheezing (a whistling sound when breathing out)
- Coughing, especially at night or early in the morning
- Shortness of breath
- Tightness in the chest

Question 5:
How can asthma be managed?
Answer: Asthma can be managed through various ways, including:
- Avoiding triggers, such as allergens and irritants
- Taking prescribed medications, such as inhalers or oral medications
- Regularly monitoring and keeping track of asthma symptoms
- Following an asthma action plan, which includes instructions on how to use medications and when to seek medical help

Question 6:
What are some common triggers for asthma?
Answer: Common triggers for asthma include:
- Allergens, such as pollen, dust mites, and pet dander
- Irritants, such as smoke, strong odors,


In [33]:
# Clean up unnecessary metadata (e.g., "Español", "January", etc.)
bad_phrases = ["Español", "January", "Exercise", "Solution", "Follow-up", "Symptoms Symptoms", "Key points"]
for bad_word in bad_phrases:
    retrieved_content = retrieved_content.replace(bad_word, "")
retrieved_content = retrieved_content.strip()

contextual_prompt = (
    "You are a medical expert. Answer this one following question directly and factually. Do not include any context, explanations, or examples in your answer.\n"
    f"Context:\n{retrieved_content}\n\n"
    f"Question: {prompt}\n"
    "Answer:"
)

# Generate the response
response = generator(
    contextual_prompt,
    max_new_tokens=200,  # Adjust as needed to capture the full answer
    do_sample=False,     # Ensures a deterministic response
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

# Extract the generated text
generated_text = response[0]['generated_text']

# Clean the output to get only the answer part
answer = generated_text.split("Answer:")[-1].strip()  # After "Answer:", the actual answer should follow

# Print only the answer
print(generated_text)


You are a medical expert. Only answer this one following question directly and factually. Do not include any context, explanations, or examples in your answer.
Context:
22, 2024
EspaÃ±ol
About Asthma

Asthma is a disease that affects your lungs.
It is one of the most common long-term diseases of children, but adults can have asthma, too.
Asthma caus

Question: What are the symptoms of asthma? in bullet points
Answer:
- Coughing
- Wheezing
- Shortness of breath
- Chest tightness



In [36]:
# Clean the output to get only the answer part
answer = generated_text.split("Answer:")[1].strip()  # After "Answer:", the actual answer should follow

# Print only the answer
print(answer)


- Coughing
- Wheezing
- Shortness of breath
- Chest tightness
